# 🍎 과일 값 계산

### 💠 역전파 노드

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, dout):
        """
        out = x * y, dout = out의 기울기
        
        dx를 구하기 위해
            1. out을 x에 대해 편미분 -> y
            2. 1.의 값에 dout을 곱함 -> dout * y
        
        dy도 대칭적으로 같은 방법을 사용
        """
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [2]:
class AddLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x + y
    
    def backward(self, dout):
        """
        out = x + y, dout = out의 기울기
        
        dx를 구하기 위해
            1. out을 x에 대해 편미분 -> 1
            2. 1.의 값에 dout을 곱함 -> dout * 1
        
        dy도 대칭적으로 같은 방법을 사용
        """
        dx = dout * 1
        dy = dout * 1
        return dx, dy

### 🍎 사과 값 계산

영수증에 찍힌 총 지출과 세율을 알 때, 사과 하나의 가격을 추측해봅시다.

In [127]:
"""
총 지출액 = apple * apple_count * tax
                  ^             ^
                  layer1        layer2
"""
total = 221 # 총 지출액

apple = 100
count = 2
tax = 1.1
# ^ 우선 이정도겠지~ 하고 적당히 설정

layer1 = MulLayer()
layer2 = MulLayer()

# 순전파
apples = layer1.forward(apple, count)
pred = layer2.forward(apples, tax)
print(f'추측한 총 지출액\t: {pred:.2f}')
print(f'실제 총 지출액\t\t: {total:.2f}')


# 역전파
error = total - pred
d_apples, d_tax = layer2.backward(error)
d_apple, d_count = layer1.backward(d_apples)
print(
    f'\n=== 역전파 ===\n'
    f'{error=:.2f}\n'
    f'=> {d_apples=:.2f}\n'
    f'   {d_tax=:.2f}\n'
    f'   => {d_apple=:.2f}\n'
    f'      {d_count=:.2f}\n'
)

# 0.1배 하여 사과의 가격에 반영
apple = apple + d_apple * 0.1
apples = layer1.forward(apple, count)
pred = layer2.forward(apples, tax)
print(f'조정한 총 지출액\t: {pred:.2f}')

추측한 총 지출액	: 220.00
실제 총 지출액		: 221.00

=== 역전파 ===
error=1.00
=> d_apples=1.10
   d_tax=200.00
   => d_apple=2.20
      d_count=110.00

조정한 총 지출액	: 220.48


### 🍊 그럼... 오렌지도 같이?
~~탕~탕~ 후루! 후루!~~

In [128]:
"""
apples = apple * apple_count
               ^
               layer1

oranges = orange * orange_count
                 ^
                 layer2

총 지출액 = (apples + oranges) * tax
                    ^          ^
                    layer3     layer4
"""
total = 716
apple = 100
apple_count = 2
orange = 150
orange_count = 3
tax = 1.1
# ^ 우선 이정도겠지~ 하고 적당히 설정

layer1 = MulLayer()
layer2 = MulLayer()
layer3 = AddLayer()
layer4 = MulLayer()

# 순전파
apples = layer1.forward(apple, apple_count)
oranges = layer2.forward(orange, orange_count)
apples_and_oranges = layer3.forward(apples, oranges)
pred = layer4.forward(apples_and_oranges, tax)
print(f'추측한 총 지출액\t: {pred:.2f}')
print(f'실제 총 지출액\t\t: {total:.2f}')


# 역전파
error = total - pred
d_apples_and_oranges, d_tax = layer4.backward(error)
d_apples, d_oranges = layer3.backward(d_apples_and_oranges)
d_orange, d_orange_count = layer2.backward(d_oranges)
d_apple, d_apple_count = layer1.backward(d_apples)

print(
    f'\n# 역전파\n'
    f'\t\t{error=:.2f}\n'
    f'{d_tax=:.2f}\t{d_apples_and_oranges=:.2f}\n'
    f'\t\t{d_apples=:.2f}\t\t{d_oranges=:.2f}\n'
    f'{d_apple=:.2f}\t{d_apple_count=:.2f}\t{d_orange=:.2f}\t{d_orange_count=:.2f}\n'
)

# 0.1배 하여 과일 가격에 반영
apple = apple + d_apple * 0.1
orange = orange + d_orange * 0.1

apples = layer1.forward(apple, apple_count)
oranges = layer2.forward(orange, orange_count)
apples_and_oranges = layer3.forward(apples, oranges)
pred = layer4.forward(apples_and_oranges, tax)
print(f'조정한 총 지출액\t: {pred:.2f}')

추측한 총 지출액	: 715.00
실제 총 지출액		: 716.00

# 역전파
		error=1.00
d_tax=650.00	d_apples_and_oranges=1.10
		d_apples=1.10		d_oranges=1.10
d_apple=2.20	d_apple_count=110.00	d_orange=3.30	d_orange_count=165.00

조정한 총 지출액	: 716.57
